In [1]:
# imports
import pandas as pd


In [2]:
# read data
df = pd.read_csv('C:/Users/Manoj Yadav/Desktop/Part II/ML II/Recommender system/BX-Books.csv', usecols=[0,1,2,3,4], encoding = "ISO-8859-1", low_memory=False)
# load ratings
rt = pd.read_csv('C:/Users/Manoj Yadav/Desktop/Part II/ML II/Recommender system/BX-Books-Ratings.csv', encoding = "ISO-8859-1", low_memory=False)


In [3]:
# get count & mean of ratings
rt = rt.groupby('ISBN')['Book-Rating'].agg(['count','mean'])
rt = rt.reset_index()
rt.columns = ['ISBN','RatingsCount','RatingsAverage']


In [4]:
# merge cols based on key, inner left
print("Rows:",len(df))
df = pd.merge(df, rt, on='ISBN', how='left')
print("Rows:",len(df))
print(df.head())


Rows: 271379
Rows: 271379
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp, Company   

   RatingsCount  RatingsAverage  
0           1.0        0.000000  
1          14.0        4.928571  
2           

In [5]:
# type
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
 #shape
print(df.shape)

(271379, 7)


In [7]:
# info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271379 entries, 0 to 271378
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 271379 non-null  object 
 1   Book-Title           271379 non-null  object 
 2   Book-Author          271378 non-null  object 
 3   Year-Of-Publication  271379 non-null  object 
 4   Publisher            271377 non-null  object 
 5   RatingsCount         270170 non-null  float64
 6   RatingsAverage       270170 non-null  float64
dtypes: float64(2), object(5)
memory usage: 16.6+ MB
None


In [8]:
# describe
print(df.describe())

        RatingsCount  RatingsAverage
count  270170.000000   270170.000000
mean        3.816764        2.884259
std        13.811221        3.284504
min         1.000000        0.000000
25%         1.000000        0.000000
50%         1.000000        1.818182
75%         3.000000        5.000000
max      2502.000000       10.000000


In [9]:
# head
print(df.head())


         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp, Company   

   RatingsCount  RatingsAverage  
0           1.0        0.000000  
1          14.0        4.928571  
2           3.0        5.000000  
3   

In [10]:
# C is the mean vote across the whole report
C = df['RatingsAverage'].mean()
print(C)

2.884258615230613


In [11]:
# m is the minimum votes required to be listed in the chart
m = df['RatingsCount'].quantile(0.70)
print(m)

2.0


In [12]:
# create a copy and filter for m
# Filter out all qualified movies into a new DataFrame
tm = df.copy()
print("Rows:",len(tm))
tm = tm[tm['RatingsCount'] >= m]
print(type(tm))
print("Rows:",len(tm))
print(tm.head())

Rows: 271379
<class 'pandas.core.frame.DataFrame'>
Rows: 124512
         ISBN                                         Book-Title  \
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
5  0399135782                             The Kitchen God's Wife   
6  0425176428  What If?: The World's Foremost Military Histor...   

            Book-Author Year-Of-Publication                 Publisher  \
1  Richard Bruce Wright                2001     HarperFlamingo Canada   
2          Carlo D'Este                1991           HarperPerennial   
3      Gina Bari Kolata                1999      Farrar Straus Giroux   
5               Amy Tan                1991          Putnam Pub Group   
6         Robert Cowley                2000  Berkley Publishing Group   

   RatingsCount  RatingsAverage  
1          14.0        4.928571  
2           3.0     

In [13]:
# function that computes the weighted rating of each movie
def weightedRating(x, m=m, C=C):
    v = x['RatingsCount']
    R = x['RatingsAverage']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
# define a new feature 'score' and calculate its value with `weighted_rating()`
tm['score'] = tm.apply(weightedRating, axis=1)
print(type(tm))
print(tm.shape)
print(tm.head())

<class 'pandas.core.frame.DataFrame'>
(124512, 8)
         ISBN                                         Book-Title  \
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
5  0399135782                             The Kitchen God's Wife   
6  0425176428  What If?: The World's Foremost Military Histor...   

            Book-Author Year-Of-Publication                 Publisher  \
1  Richard Bruce Wright                2001     HarperFlamingo Canada   
2          Carlo D'Este                1991           HarperPerennial   
3      Gina Bari Kolata                1999      Farrar Straus Giroux   
5               Amy Tan                1991          Putnam Pub Group   
6         Robert Cowley                2000  Berkley Publishing Group   

   RatingsCount  RatingsAverage     score  
1          14.0        4.928571  4.673032  
2           3.

In [15]:
# print the top 15 movies
print(tm[['Book-Title', 'RatingsCount', 'RatingsAverage', 'score']].head(15))



                                           Book-Title  RatingsCount  \
1                                        Clara Callan          14.0   
2                                Decision in Normandy           3.0   
3   Flu: The Story of the Great Influenza Pandemic...          11.0   
5                              The Kitchen God's Wife          33.0   
6   What If?: The World's Foremost Military Histor...           5.0   
7                                     PLEADING GUILTY           3.0   
10                        Nights Below Station Street           6.0   
12                                 The Middle Stories           2.0   
14  A Second Chicken Soup for the Woman's Soul (Ch...           8.0   
16  More Cunning Than Man: A Social History of Rat...           3.0   
17                      Goodbye to the Buttermilk Sky           3.0   
18                                      The Testament         422.0   
19               Beloved (Plume Contemporary Fiction)         180.0   
20  Ou

In [18]:
# movies of a genre
def getBooksByAuthor(pAuthor):
    global tm
    #pAuthor = 'Gina Bari Kolata'
    mask = tm['Book-Author'].apply(lambda x: pAuthor in x)
    mg = tm[mask]
    # print the top 5 movies
    print("\nAuthor:", pAuthor)
    print("==============================================")
    print(mg[['Book-Title', 'RatingsCount', 'RatingsAverage', 'score']].head(5))



In [17]:
# sample genre
getBooksByAuthor('Gina Bari Kolata')
getBooksByAuthor('Amy Tan')



Author: Gina Bari Kolata
                                          Book-Title  RatingsCount  \
3  Flu: The Story of the Great Influenza Pandemic...          11.0   

   RatingsAverage     score  
3        4.272727  4.059117  

Author: Amy Tan
                                           Book-Title  RatingsCount  \
5                              The Kitchen God's Wife          33.0   
67                                  The Joy Luck Club         519.0   
1429                           The Kitchen God's Wife         273.0   
2377  The Kitchen God's Wife (Vintage Contemporaries)          13.0   
2969                        The Bonesetter's Daughter         117.0   

      RatingsAverage     score  
5           4.212121  4.136243  
67          3.063584  3.062895  
1429        2.875458  2.875522  
2377        4.923077  4.651234  
2969        3.358974  3.350996  


In [19]:
lstAuthor = (tm['Book-Author'].unique())
print(lstAuthor)
print(len(lstAuthor))

['Richard Bruce Wright' "Carlo D'Este" 'Gina Bari Kolata' ...
 'David Bailey' 'Dody Myers' 'Frances Towner Giedt']
43589
